In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
energy_df = pd.read_csv('energydata_complete.csv')

In [4]:
energy_df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [9]:
df = energy_df.drop(['date', 'lights'], axis = 1)


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [11]:
X = normalized_df.drop(['Appliances'], axis = 1)
y = normalized_df['Appliances']


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [13]:
x1 = np.array(normalized_df['T2'])
y1 = np.array(normalized_df['T6'])
#we need to reshape the x variable because we have only one predictor here
x1 = x1.reshape(-1, 1)
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size = 0.3, random_state = 42)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x1_train, y1_train)
y1_pred = model.predict(x1_test)

from sklearn.metrics import r2_score
r2 = r2_score(y1_test, y1_pred)
round(r2, 2)


0.64

In [15]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
train_score = reg.score(X_train, y_train)
test_score = reg.score(X_test, y_test)
print("Training score of linear regression model is {:.3f}".format(train_score))
print("Testing score of linear regression model is {:.3f}".format(test_score))


Training score of linear regression model is 0.145
Testing score of linear regression model is 0.149


In [16]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error for linear model: {}'.format(round(mae, 2)))


Mean Absolute Error for linear model: 0.05


In [17]:
rss = np.sum(np.square(y_test - y_pred))
print('Residual Sum of Squares for linear model: {}'.format(round(rss, 2)))

Residual Sum of Squares for linear model: 45.35


In [18]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('Root Mean Squared Error for linear model: {}'.format(round(rmse, 3)))


Root Mean Squared Error for linear model: 0.088


In [19]:
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, y_pred)
print('R_Squared for linear model: {}'.format(round(r_squared, 2)))


R_Squared for linear model: 0.15


In [20]:
def weights(model, feat, col_name):
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_reg_weights = weights(reg, X_train, 'Linear_model_Weight')
linear_reg_weights


,Features,Linear_model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [21]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.4)
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
train_score = ridge.score(X_train, y_train)
test_score = ridge.score(X_test, y_test)
print("Training score of ridge regression model is {:.3f}".format(train_score))
print("Testing score of ridge regression model is {:.3f}".format(test_score))


Training score of ridge regression model is 0.145
Testing score of ridge regression model is 0.149


In [22]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, ridge_pred)
rmse = np.sqrt(mse)
print('Root Mean Squared Error for ridge model: {}'.format(round(rmse, 3)))



Root Mean Squared Error for ridge model: 0.088


In [23]:
from sklearn.linear_model import Lasso
lasso0001 = Lasso(alpha = 0.001)
lasso0001.fit(X_train, y_train)
lasso0001_pred = lasso0001.predict(X_test)
train_score = lasso0001.score(X_train, y_train)
test_score = lasso0001.score(X_test, y_test)
print("Training score of lasso regression model is {:.3f}".format(train_score))
print("Testing score of lasso regression model is {:.3f}".format(test_score))
print("Number of features for training: {:.3f}".format(np.sum(lasso0001.coef_ != 0)))


Training score of lasso regression model is 0.025
Testing score of lasso regression model is 0.027
Number of features for training: 4.000


In [24]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, lasso0001_pred)
rmse = np.sqrt(mse)
print('Root Mean Squared Error for ridge model: {}'.format(round(rmse, 3)))


Root Mean Squared Error for ridge model: 0.094
